In [38]:
from db_analysis import read_gt, visualize, stats, sign_counter
from color_analysis import blue_test, color_segmentation_hsv, color_analysis, parse_sign_type
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data loading and Analysis

In [41]:
gt_train_dir  = "./train/gt/"
im_train_dir  = "./train/"
msk_train_dir = "./train/mask/"

gt = read_gt(gt_train_dir)

print('Train data size: ', len(gt))
print('First example [image id, [tly, tlx, bry, brx], sign type, aspect ratio] :')
print(gt[0])

keys = [s[0] for s in sign_count[:]]
data = {'Sign count':     [s[1] for s in sign_count[:]],
        'filling ratios': [sum(filling_ratios[key])/len(filling_ratios[key])  for key in  keys] }
keys_name = [key+' - '+parse_sign_type(key) for key in keys]

df = pd.DataFrame(data=data, index = keys_name)
df

Train data size:  393
First example [image id, [tly, tlx, bry, brx], sign type, aspect ratio] :
['00.004767', [94.9, 1410.99, 150.3, 1468.95], 'F', 1.046209386281589]


,Sign count,filling ratios
D - Blue Circle,71,0.779416
F - Blue Square,120,1.000000
C - Red Circle,47,0.783965
A - Red Triangle,103,0.500770
E - Red and Blue Circle,38,0.784881
B - Yield,14,0.496148
